In [ ]:
def loss_augment_unaries(unary_potentials, y,class_weight):
    n_states = unary_potentials.shape[1]
    for i in range(unary_potentials.shape[0]):
        for s in range(n_states):
            if s == y[i]:
                continue
            unary_potentials[i, s] += class_weight[y[i]]
def lp_general_graph(unaries, edges, edge_weights):
    if unaries.shape[1] != edge_weights.shape[1]:
        raise ValueError("incompatible shapes of unaries"
                         " and edge_weights.")
    if edge_weights.shape[1] != edge_weights.shape[2]:
        raise ValueError("Edge weights not square!")
    if edge_weights.shape[0] != edges.shape[0]:
        raise ValueError("Number of edge weights different from number of"
                         "edges")

    n_nodes, n_states = map(int, unaries.shape)
    n_edges = len(edges)

    # variables: n_nodes * n_states for nodes,
    # n_edges * n_states ** 2 for edges
    n_variables = n_nodes * n_states + n_edges * n_states ** 2

    # constraints: one per node,
    # and n_nodes * n_states for pairwise minus one redundant per edge
    n_constraints = n_nodes + n_edges * (2 * n_states - 1)

    # offset to get to the edge variables in columns
    edges_offset = n_nodes * n_states
    # store constraints as triple (data, I, J)
    data, I, J = [], [], []

    # summation constraints
    for i in range(n_nodes):
        for j in range(n_states):
            data.append(1)
            I.append(i)
            J.append(i * n_states + j)
            #constraints[i, i * n_states + j] = 1
    # we row_idx tracks constraints = rows in constraint matrix
    row_idx = n_nodes
    # edge marginalization constraint
    for i in range(2 * n_edges * n_states):
        edge = i // (2 * n_states)
        state = (i % n_states)
        vertex_in_edge = i % (2 * n_states) // n_states
        vertex = edges[edge][vertex_in_edge]
        if vertex_in_edge == 1 and state == n_states - 1:
            # the last summation constraint is redundant.
            continue
        # for one vertex iterate over all states of the other vertex
        #[row_idx, int(vertex) * n_states + state] = -1
        data.append(-1)
        I.append(row_idx)
        J.append(int(vertex) * n_states + state)
        edge_var_index = edges_offset + edge * n_states ** 2
        if vertex_in_edge == 0:
            # first vertex in edge
            for j in range(n_states):
                data.append(1)
                I.append(row_idx)
                J.append(edge_var_index + state * n_states + j)
                #[row_idx, edge_var_index + state * n_states + j] = 1
        else:
            # second vertex in edge
            for j in range(n_states):
                data.append(1)
                I.append(row_idx)
                J.append(edge_var_index + j * n_states + state)
                #[row_idx, edge_var_index + j * n_states + state] = 1
        row_idx += 1

    coef = np.ravel(unaries)
    # pairwise:
    repeated_pairwise = edge_weights.ravel()
    coef = np.hstack([coef, repeated_pairwise])
    c = cvxopt.matrix(coef, tc='d')
    # for positivity inequalities
    G = cvxopt.spdiag(cvxopt.matrix(-np.ones(n_variables)))
    #G = cvxopt.matrix(-np.eye(n_variables))
    h = cvxopt.matrix(np.zeros(n_variables))  # for positivity inequalities
    # unary and pairwise summation constratints
    A = cvxopt.spmatrix(data, I, J)
    assert(n_constraints == A.size[0])
    b_ = np.zeros(A.size[0])  # zeros for pairwise summation constraints
    b_[:n_nodes] = 1    # ones for unary summation constraints
    b = cvxopt.matrix(b_)

    # don't be verbose.
    show_progress_backup = cvxopt.solvers.options.get('show_progress', False)
    cvxopt.solvers.options['show_progress'] = False
    result = cvxopt.solvers.lp(c, G, h, A, b)
    cvxopt.solvers.options['show_progress'] = show_progress_backup

    x = np.array(result['x'])
    unary_variables = x[:n_nodes * n_states].reshape(n_nodes, n_states)
    pairwise_variables = x[n_nodes * n_states:].reshape(n_edges, n_states ** 2)
    assert((np.abs(unary_variables.sum(axis=1) - 1) < 1e-4).all())
    assert((np.abs(pairwise_variables.sum(axis=1) - 1) < 1e-4).all())
    return unary_variables, pairwise_variables, result['primal objective']

def _validate_params(unary_potentials, pairwise_params, edges):
    n_states = unary_potentials.shape[-1]
    if pairwise_params.shape == (n_states, n_states):
        # only one matrix given
        pairwise_potentials = np.repeat(pairwise_params[np.newaxis, :, :],
                                        edges.shape[0], axis=0)
    else:
        if pairwise_params.shape != (edges.shape[0], n_states, n_states):
            raise ValueError("Expected pairwise_params either to "
                             "be of shape n_states x n_states "
                             "or n_edges x n_states x n_states, but"
                             " got shape %s" % repr(pairwise_params.shape))
        pairwise_potentials = pairwise_params
    return n_states, pairwise_potentials

def crammer_singer_joint_feature(X, Y,out):
    for i in range(X.shape[0]):
        y = Y[i]
        for j in range(X.shape[1]):
            out[y, j] += X[i, j]
def get_installed(method_filter=None):
    if method_filter is None:
        method_filter = ["max-product", 'ad3', 'ad3+', 'qpbo', 'ogm', 'lp']

    installed = []
    unary = np.zeros((1, 1))
    pw = np.zeros((1, 1))
    edges = np.empty((0, 2), dtype=np.int)
    for method in method_filter:
        try:
            if method != 'ad3+':
                inference_dispatch(unary, pw, edges, inference_method=method)
            else:
                inference_dispatch(unary, np.zeros((0,1,1))
                                   , np.zeros((0,2), dtype=np.int)
                                   , inference_method=method)
            installed.append(method)
        except ImportError:
            pass
    return installed
def inference_dispatch(unary_potentials, pairwise_potentials, edges,
                       inference_method, return_energy=False, **kwargs):
    """
    Computes the maximizing assignment of a pairwise discrete energy function.
    Wrapper function to dispatch between inference method by string.
    Parameters
    ----------
    unary_potentials : nd-array, shape (n_nodes, n_states)
        Unary potentials of energy function.
    pairwise_potentials : nd-array, shape (n_states, n_states) 
                        or (n_states, n_states, n_edges).
        Pairwise potentials of energy function.
        If the first case, edge potentials are assumed to be the same for all 
        edges.
        In the second case, the sequence needs to correspond to the edges.
    edges : nd-array, shape (n_edges, 2)
        Graph edges for pairwise potentials, given as pair of node indices. As
        pairwise potentials are not assumed to be symmetric, the direction of
        the edge matters.
    inference_method : string
        Possible choices currently are:
            * 'qpbo' for QPBO alpha-expansion (fast but approximate).
            * 'lp' for build-in lp relaxation via cvxopt (slow).
            * 'ad3' for AD^3 subgradient based dual solution of LP.
            * 'ogm' for OpenGM wrappers.
            * 'max-product' for max-product message passing.
            * 'unary' for using unary potentials only.
        It is also possible to pass a tuple (string, dict) where the dict
        contains additional keyword arguments, like
        ``('ad3', {'branch_and_bound': True})``.
    relaxed : bool (default=False)
        Whether to return a relaxed solution (when appropriate)
        or round to the nearest integer solution. Only used for 'lp' and 'ad3'
        inference methods.
    return_energy : bool (default=False)
        Additionally return the energy of the returned solution (according to
        the solver).  If relaxed=False, this is the energy of the relaxed, not
        the rounded solution.
    Returns
    -------
    labels : nd-array
        Approximate (usually) MAP variable assignment.
        If relaxed=True, this is a tuple of unary and pairwise "marginals"
        from the LP relaxation.
    """
    if isinstance(inference_method, tuple):
        additional_kwargs = inference_method[1]
        inference_method = inference_method[0]
        # append additional_kwargs, but take care not to modify the dicts we
        # got
        kwargs = kwargs.copy()
        kwargs.update(additional_kwargs)
    if inference_method == "qpbo":
        return inference_qpbo(unary_potentials, pairwise_potentials, edges,
                              **kwargs)
    elif inference_method == "lp":
        return inference_lp(unary_potentials, pairwise_potentials, edges,
                            return_energy=return_energy, **kwargs)
    elif inference_method == "ad3":
        return inference_ad3(unary_potentials, pairwise_potentials, edges,
                             return_energy=return_energy, **kwargs)
    elif inference_method == "ad3+":
        return inference_ad3plus(unary_potentials, pairwise_potentials, edges,
                             return_energy=return_energy, **kwargs)
    elif inference_method == "ogm":
        return inference_ogm(unary_potentials, pairwise_potentials, edges,
                             return_energy=return_energy, **kwargs)
    elif inference_method == "unary":
        return inference_unaries(unary_potentials, pairwise_potentials, edges,
                                 **kwargs)
    elif inference_method == "max-product":
        return inference_max_product(unary_potentials, pairwise_potentials,
                                     edges, **kwargs)
    else:
        raise ValueError("inference_method must be 'max-product', 'lp', 'ad3',"
                         " 'qpbo' or 'ogm', got %s" % inference_method)


def inference_ogm(unary_potentials, pairwise_potentials, edges,
                  return_energy=False, alg='dd', init=None,
                  reserveNumFactorsPerVariable=2, **kwargs):
    """Inference with OpenGM backend.
    Parameters
    ----------
    unary_potentials : nd-array, shape (n_nodes, n_states)
        Unary potentials of energy function.
    pairwise_potentials : nd-array, shape (n_states, n_states) 
                        or (n_states, n_states, n_edges).
        Pairwise potentials of energy function.
        If the first case, edge potentials are assumed to be the same for all 
        edges.
        In the second case, the sequence needs to correspond to the edges.
    edges : nd-array, shape (n_edges, 2)
        Graph edges for pairwise potentials, given as pair of node indices. As
        pairwise potentials are not assumed to be symmetric, the direction of
        the edge matters.
    alg : string
        Possible choices currently are:
            * 'bp' for Loopy Belief Propagation.
            * 'dd' for Dual Decomposition via Subgradients.
            * 'trws' for Vladimirs TRWs implementation.
            * 'trw' for OGM  TRW.
            * 'gibbs' for Gibbs sampling.
            * 'lf' for Lazy Flipper
            * 'fm' for Fusion Moves (alpha-expansion fusion)
            * 'dyn' for Dynamic Programming (message passing in trees)
            * 'gc' for Graph Cut
            * 'alphaexp' for Alpha Expansion using Graph Cuts
            * 'mqpbo' for multi-label qpbo
    init : nd-array
        Initial solution for starting inference (ignored by some algorithms).
    reserveNumFactorsPerVariable :
        reserve a certain number of factors for each variable can speed up
        the building of a graphical model.
        ( For a 2d grid with second order factors one should set this to 5
         4 2-factors and 1 unary factor for most pixels )
    Returns
    -------
    labels : nd-array
        Approximate (usually) MAP variable assignment.
    """

    import opengm
    n_states, pairwise_potentials = \
        _validate_params(unary_potentials, pairwise_potentials, edges)
    n_nodes = len(unary_potentials)

    gm = opengm.gm(np.ones(n_nodes, dtype=opengm.label_type) * n_states)

    nFactors = int(n_nodes + edges.shape[0])
    gm.reserveFactors(nFactors)
    gm.reserveFunctions(nFactors, 'explicit')

    # all unaries as one numpy array
    # (opengm's value_type == float64 but all types are accepted)
    unaries = np.require(unary_potentials, dtype=opengm.value_type) * -1.0
    # add all unart functions at once
    fidUnaries = gm.addFunctions(unaries)
    visUnaries = np.arange(n_nodes, dtype=opengm.label_type)
    # add all unary factors at once
    gm.addFactors(fidUnaries, visUnaries)

    # add all pariwise functions at once
    # - first axis of secondOrderFunctions iterates over the function)

    secondOrderFunctions = -np.require(pairwise_potentials,
                                       dtype=opengm.value_type)
    fidSecondOrder = gm.addFunctions(secondOrderFunctions)
    gm.addFactors(fidSecondOrder, edges.astype(np.uint64))

    if alg == 'bp':
        inference = opengm.inference.BeliefPropagation(gm)
    elif alg == 'dd':
        inference = opengm.inference.DualDecompositionSubgradient(gm)
    elif alg == 'trws':
        inference = opengm.inference.TrwsExternal(gm)
    elif alg == 'trw':
        inference = opengm.inference.TreeReweightedBp(gm)
    elif alg == 'gibbs':
        inference = opengm.inference.Gibbs(gm)
    elif alg == 'lf':
        inference = opengm.inference.LazyFlipper(gm)
    elif alg == 'icm':
        inference = opengm.inference.Icm(gm)
    elif alg == 'dyn':
        inference = opengm.inference.DynamicProgramming(gm)
    elif alg == 'fm':
        inference = opengm.inference.AlphaExpansionFusion(gm)
    elif alg == 'gc':
        inference = opengm.inference.GraphCut(gm)
    elif alg == 'loc':
        inference = opengm.inference.Loc(gm)
    elif alg == 'mqpbo':
        inference = opengm.inference.Mqpbo(gm)
    elif alg == 'alphaexp':
        inference = opengm.inference.AlphaExpansion(gm)
    if init is not None:
        inference.setStartingPoint(init)

    inference.infer()
    # we convert the result to int from unsigned int
    # because otherwise we are sure to shoot ourself in the foot
    res = inference.arg().astype(np.int)
    if return_energy:
        return res, gm.evaluate(res)
    return res
def inference_ad3(unary_potentials, pairwise_potentials, edges, relaxed=False,
                  verbose=0, return_energy=False, branch_and_bound=False,
                  inference_exception=None):
    """Inference with AD3 dual decomposition subgradient solver.
    Parameters
    ----------
    unary_potentials : nd-array, shape (n_nodes, n_states)
        Unary potentials of energy function.
    pairwise_potentials : nd-array, shape (n_states, n_states) 
                        or (n_states, n_states, n_edges).
        Pairwise potentials of energy function.
        If the first case, edge potentials are assumed to be the same for all 
        edges.
        In the second case, the sequence needs to correspond to the edges.
    edges : nd-array, shape (n_edges, 2)
        Graph edges for pairwise potentials, given as pair of node indices. As
        pairwise potentials are not assumed to be symmetric, the direction of
        the edge matters.
    relaxed : bool (default=False)
        Whether to return the relaxed solution (``True``) or round to the next
        integer solution (``False``).
    verbose : int (default=0)
        Degree of verbosity for solver.
    return_energy : bool (default=False)
        Additionally return the energy of the returned solution (according to
        the solver).  If relaxed=False, this is the energy of the relaxed, not
        the rounded solution.
    branch_and_bound : bool (default=False)
        Whether to attempt to produce an integral solution using
        branch-and-bound.
    Returns
    -------
    labels : nd-array
        Approximate (usually) MAP variable assignment.
        If relaxed=False, this is a tuple of unary and edge 'marginals'.
        
    Code updated on Feb 2017 to deal with multiple node types, by JL Meunier
    , for the EU READ project (grant agreement No 674943)
    
    """
    import ad3
    bMultiType = isinstance(unary_potentials, list)
    if bMultiType:
        res = ad3.general_graph(unary_potentials, edges, pairwise_potentials
                                , verbose=verbose
                                , n_iterations=4000, exact=branch_and_bound)
    else:
        #usual code
        n_states, pairwise_potentials = \
            _validate_params(unary_potentials, pairwise_potentials, edges)
        unaries = unary_potentials.reshape(-1, n_states)
        res = ad3.general_graph(unaries, edges, pairwise_potentials
                                , verbose=verbose, n_iterations=4000
                                , exact=branch_and_bound)
        
    unary_marginals, pairwise_marginals, energy, solver_status = res
    if verbose:
        print(solver_status)

    if solver_status in ["fractional", "unsolved"] and relaxed:
        if bMultiType:
            y = (unary_marginals, pairwise_marginals)  #those two are lists
        else:
            #usual code
            unary_marginals = unary_marginals.reshape(unary_potentials.shape)
            y = (unary_marginals, pairwise_marginals)
        #print solver_status, pairwise_marginals
    else:
        if bMultiType:
            #we now get a list of unary marginals
            if inference_exception and solver_status in ["fractional"
                                                         , "unsolved"]:
                raise InferenceException(solver_status)
            ly = list()
            _cum_n_states = 0
            for unary_marg in unary_marginals:
                ly.append( _cum_n_states + np.argmax(unary_marg, axis=-1) )
                # number of states for that type
                _cum_n_states += unary_marg.shape[1]  
            y = np.hstack(ly)            
        else:
            #usual code
            y = np.argmax(unary_marginals, axis=-1)
        
    if return_energy:
        return y, -energy
    return y


def inference_ad3plus(l_unary_potentials, l_pairwise_potentials, l_edges
                      , relaxed=False
                      , verbose=0, return_energy=False, branch_and_bound=False
                      , constraints=None, inference_exception=None):
    """
    Inference with AD3 dual decomposition subgradient solver.
    Parameters
    ----------
    unary_potentials : nd-array, shape (n_nodes, n_states)
        Unary potentials of energy function.
    pairwise_potentials : nd-array, shape (n_states, n_states) 
                        or (n_states, n_states, n_edges).
        Pairwise potentials of energy function.
        If the first case, edge potentials are assumed to be the same for all 
        edges.
        In the second case, the sequence needs to correspond to the edges.
    edges : nd-array, shape (n_edges, 2)
        Graph edges for pairwise potentials, given as pair of node indices. As
        pairwise potentials are not assumed to be symmetric, the direction of
        the edge matters.
    relaxed : bool (default=False)
        Whether to return the relaxed solution (``True``) or round to the next
        integer solution (``False``).
    verbose : int (default=0)
        Degree of verbosity for solver.
    return_energy : bool (default=False)
        Additionally return the energy of the returned solution (according to
        the solver).  If relaxed=False, this is the energy of the relaxed, not
        the rounded solution.
    branch_and_bound : bool (default=False)
        Whether to attempt to produce an integral solution using
        branch-and-bound.
    constraints : list of logical constraints or None (default:=None)
        A logical constraint is tuple like 
            ( <operator>, <unaries>, <states>, <negated> )
        where:
        - operator is one of:
             'XOR' 'XOROUT' 'ATMOSTONE' 'OR' 'OROUT' 'ANDOUT' 'IMPLY'
        - unaries is a list of the index of each unary involved in this 
        constraint
        - states is a list of unary states (class), 1 per involved unary. If the
         states are all the same, you can pass it directly as a scalar value.
        - negated is a list of boolean indicating if the unary must be negated.
        Again, if all values are the same, pass a single boolean value instead
        of a list 
        
        NOTE: this hard logic constraint mechanism has been developed for the 
        EU project READ, by JL Meunier (Xerox), in November 2016.
        The READ project has received funding from the European Union's Horizon
        2020 research and innovation programme under grant agreement No 674943.
    
    Returns
    -------
    labels : nd-array
        Approximate (usually) MAP variable assignment.
        If relaxed=False, this is a tuple of unary and edge 'marginals'.
    """
    import ad3
#     n_states, pairwise_potentials = \
#         _validate_params(unary_potentials, pairwise_potentials, edges)
#     unaries = unary_potentials.reshape(-1, n_states)
    bMultiType = isinstance(l_unary_potentials, list)

    res = ad3.general_constrained_graph(l_unary_potentials, l_edges
                                        , l_pairwise_potentials, constraints
                                        , verbose=verbose
                                        , n_iterations=4000
                                        , exact=branch_and_bound)
    
    l_unary_marginals, l_pairwise_marginals, energy, solver_status = res
    if verbose:
        print(solver_status)

    if relaxed and solver_status in ["fractional", "unsolved"]:
        y = (l_unary_marginals, l_pairwise_marginals)
    else:
        if inference_exception and solver_status in ["fractional", "unsolved"]:
            raise InferenceException(solver_status)
        if bMultiType:
            #we now get a list of unary marginals
            ly = list()
            _cum_n_states = 0
            for unary_marg in l_unary_marginals:
                ly.append( _cum_n_states + np.argmax(unary_marg, axis=-1) )
                #number of states for that type
                _cum_n_states += unary_marg.shape[1] 
            y = np.hstack(ly)
            # when we will simplify y: 
            #y = [_cum_n_statesnp.argmax(unary_marg, axis=-1) for unary_marg 
            #  in l_unary_marginals]
        else:
            y = np.argmax(l_unary_marginals, axis=-1)

    if return_energy:
        return y, -energy
    return y
def inference_max_product(unary_potentials, pairwise_potentials, edges,
                          max_iter=30, damping=0.5, tol=1e-5, relaxed=None):
    """Max-product inference.
    In case the edges specify a tree, dynamic programming is used
    producing a result in only a single pass.
    Parameters
    ----------
    unary_potentials : nd-array
        Unary potentials of energy function.
    pairwise_potentials : nd-array
        Pairwise potentials of energy function.
    edges : nd-array
        Edges of energy function.
    max_iter : int (default=10)
        Maximum number of iterations. Ignored if graph is a tree.
    damping : float (default=.5)
        Daming of messages in loopy message passing.
        Ignored if graph is a tree.
    tol : float (default=1e-5)
        Stopping tollerance for loopy message passing.
    """
    from ._viterbi import viterbi
    n_states, pairwise_potentials = \
        _validate_params(unary_potentials, pairwise_potentials, edges)
    if is_chain(edges=edges, n_vertices=len(unary_potentials)):
        y = viterbi(unary_potentials.astype(np.float).copy(),
                    # sad second copy b/c numpy 1.6
                    np.array(pairwise_potentials, dtype=np.float))
    elif is_forest(edges=edges, n_vertices=len(unary_potentials)):
        y = tree_max_product(unary_potentials, pairwise_potentials, edges)
    else:
        y = iterative_max_product(unary_potentials, pairwise_potentials, edges,
                                  max_iter=max_iter, damping=damping)
    return y


def tree_max_product(unary_potentials, pairwise_potentials, edges):
    n_vertices, n_states = unary_potentials.shape
    parents = -np.ones(n_vertices, dtype=np.int)
    visited = np.zeros(n_vertices, dtype=np.bool)
    neighbors = [[] for i in range(n_vertices)]
    pairwise_weights = [[] for i in range(n_vertices)]
    for pw, edge in zip(pairwise_potentials, edges):
        neighbors[edge[0]].append(edge[1])
        pairwise_weights[edge[0]].append(pw)
        neighbors[edge[1]].append(edge[0])
        pairwise_weights[edge[1]].append(pw.T)

    messages_forward = np.zeros((n_vertices, n_states))
    messages_backward = np.zeros((n_vertices, n_states))
    pw_forward = np.zeros((n_vertices, n_states, n_states))
    # build a breadth first search of the tree
    traversal = []
    lonely = 0
    while lonely < n_vertices:
        for i in range(lonely, n_vertices):
            if not visited[i]:
                queue = [i]
                lonely = i + 1
                visited[i] = True
                break
            lonely = n_vertices

        while queue:
            node = queue.pop(0)
            traversal.append(node)
            for pw, neighbor in zip(pairwise_weights[node], neighbors[node]):
                if not visited[neighbor]:
                    parents[neighbor] = node
                    queue.append(neighbor)
                    visited[neighbor] = True
                    pw_forward[neighbor] = pw

                elif not parents[node] == neighbor:
                    raise ValueError("Graph not a tree")
    # messages from leaves to root
    for node in traversal[::-1]:
        parent = parents[node]
        if parent != -1:
            message = np.max(messages_backward[node] + unary_potentials[node] +
                             pw_forward[node], axis=1)
            message -= message.max()
            messages_backward[parent] += message
    # messages from root back to leaves
    for node in traversal:
        parent = parents[node]
        if parent != -1:
            message = messages_forward[parent] + unary_potentials[parent] + pw_forward[node].T
            # leaves to root messages from other children
            message += messages_backward[parent] - np.max(messages_backward[node]
                                                          + unary_potentials[node]
                                                          + pw_forward[node], axis=1)
            message = message.max(axis=1)
            message -= message.max()
            messages_forward[node] += message

    return np.argmax(unary_potentials + messages_forward + messages_backward, axis=1)



In [ ]:
import itertools
from joblib import Parallel, delayed

import numpy as np


def unwrap_pairwise(y):
    """given a y that may contain pairwise marginals, yield plain y."""
    if isinstance(y, tuple):
        return y[0]
    return y


def expand_sym(sym_compressed):
    """Expand compressed symmetric matrix to full square matrix.

    Similar to scipy.spatial.squareform, but also contains the
    diagonal.
    """
    length = sym_compressed.size
    size = int(np.sqrt(2 * length + 0.25) - 1 / 2.)
    sym = np.zeros((size, size))
    sym[np.tri(size, dtype=np.bool)] = sym_compressed
    return (sym + sym.T - np.diag(np.diag(sym)))


def compress_sym(sym_expanded, make_symmetric=True):
    """Compress symmetric matrix to a vector.

    Similar to scipy.spatial.squareform, but also contains the
    diagonal.

    Parameters
    ----------
    sym_expanded : nd-array, shape (size, size)
        Input matrix to compress.

    make_symmetric : bool (default=True)
        Whether to symmetrize the input matrix before compressing.
        It is made symmetric by using
        ``sym_expanded + sym_expanded.T - np.diag(np.diag(sym_expanded))``
        This makes sense if only one of the two entries was non-zero before.


    """
    size = sym_expanded.shape[0]
    if make_symmetric:
        sym_expanded = (sym_expanded + sym_expanded.T -
                        np.diag(np.diag(sym_expanded)))
    return sym_expanded[np.tri(size, dtype=np.bool)]


## global functions for easy parallelization
def find_constraint(model, x, y, w, y_hat=None, relaxed=True,
                    compute_difference=True):
    """Find most violated constraint, or, given y_hat,
    find slack and djoint_feature for this constraing.

    As for finding the most violated constraint, it is enough to compute
    joint_feature(x, y_hat), not djoint_feature, we can optionally skip
    computing joint_feature(x, y) using compute_differences=False
    """

    if y_hat is None:
        y_hat = model.loss_augmented_inference(x, y, w, relaxed=relaxed)
    joint_feature = model.joint_feature
    if getattr(model, 'rescale_C', False):
        delta_joint_feature = -joint_feature(x, y_hat, y)
    else:
        delta_joint_feature = -joint_feature(x, y_hat)
    if compute_difference:
        if getattr(model, 'rescale_C', False):
            delta_joint_feature += joint_feature(x, y, y)
        else:
            delta_joint_feature += joint_feature(x, y)

    if isinstance(y_hat, tuple):
        # continuous label
        loss = model.continuous_loss(y, y_hat[0])
    else:
        loss = model.loss(y, y_hat)
    slack = max(loss - np.dot(w, delta_joint_feature), 0)
    return y_hat, delta_joint_feature, slack, loss


def find_constraint_latent(model, x, y, w, relaxed=True):
    """Find most violated constraint.

    As for finding the most violated constraint, it is enough to compute
    joint_feature(x, y_hat), not djoint_feature, we can optionally skip
    computing joint_feature(x, y) using compute_differences=False
    """
    h = model.latent(x, y, w)
    h_hat = model.loss_augmented_inference(x, h, w, relaxed=relaxed)
    joint_feature = model.joint_feature
    delta_joint_feature = joint_feature(x, h) - joint_feature(x, h_hat)

    loss = model.loss(y, h_hat)
    slack = max(loss - np.dot(w, delta_joint_feature), 0)
    return h_hat, delta_joint_feature, slack, loss


def inference(model, x, w, constraints=None):
    if constraints:
        return model.inference(x, w, constraints=constraints)
    else:
        return model.inference(x, w)


def loss_augmented_inference(model, x, y, w, relaxed=True):
    return model.loss_augmented_inference(x, y, w, relaxed=relaxed)


# easy debugging
def objective_primal(model, w, X, Y, C, variant='n_slack', n_jobs=1):
    objective = 0
    constraints = Parallel(
        n_jobs=n_jobs)(delayed(find_constraint)(
            model, x, y, w)
            for x, y in zip(X, Y))
    slacks = list(zip(*constraints))[2]

    if variant == 'n_slack':
        slacks = np.maximum(slacks, 0)

    objective = max(np.sum(slacks), 0) * C + np.sum(w ** 2) / 2.
    return objective


def exhaustive_loss_augmented_inference(model, x, y, w):
    size = y.size
    best_y = None
    best_energy = np.inf
    for y_hat in itertools.product(range(model.n_states), repeat=size):
        y_hat = np.array(y_hat).reshape(y.shape)
        #print("trying %s" % repr(y_hat))
        joint_feature = model.joint_feature(x, y_hat)
        energy = -model.loss(y, y_hat) - np.dot(w, joint_feature)
        if energy < best_energy:
            best_energy = energy
            best_y = y_hat
    return best_y


def exhaustive_inference(model, x, w):
    # hack to get the grid shape of x
    if isinstance(x, np.ndarray):
        feats = x
    else:
        feats = model._get_features(x)
    size = np.prod(feats.shape[:-1])
    best_y = None
    best_energy = np.inf
    for y_hat in itertools.product(range(model.n_states), repeat=size):
        y_hat = np.array(y_hat).reshape(feats.shape[:-1])
        #print("trying %s" % repr(y_hat))
        joint_feature = model.joint_feature(x, y_hat)
        energy = -np.dot(w, joint_feature)
        if energy < best_energy:
            best_energy = energy
            best_y = y_hat
    return best_y
def inference_lp(unary_potentials, pairwise_potentials, edges, relaxed=False,
                 return_energy=False, **kwargs):
    """Inference with build-in LP solver using cvxopt backend.
    Parameters
    ----------
    unary_potentials : nd-array, shape (n_nodes, n_states)
        Unary potentials of energy function.
    pairwise_potentials : nd-array, shape (n_states, n_states) 
                        or (n_states, n_states, n_edges).
        Pairwise potentials of energy function.
        If the first case, edge potentials are assumed to be the same for all 
        edges.
        In the second case, the sequence needs to correspond to the edges.
    edges : nd-array, shape (n_edges, 2)
        Graph edges for pairwise potentials, given as pair of node indices. As
        pairwise potentials are not assumed to be symmetric, the direction of
        the edge matters.
    relaxed : bool (default=False)
        Whether to return the relaxed solution (``True``) or round to the next
        integer solution (``False``).
    return_energy : bool (default=False)
        Additionally return the energy of the returned solution (according to
        the solver).  If relaxed=False, this is the energy of the relaxed, not
        the rounded solution.
    Returns
    -------
    labels : nd-array
        Approximate (usually) MAP variable assignment.
        If relaxed=False, this is a tuple of unary and edge 'marginals'.
    """
    shape_org = unary_potentials.shape[:-1]
    n_states, pairwise_potentials = \
        _validate_params(unary_potentials, pairwise_potentials, edges)

    unaries = unary_potentials.reshape(-1, n_states)
    res = lp_general_graph(-unaries, edges, -pairwise_potentials)
    unary_marginals, pairwise_marginals, energy = res
    if relaxed:
        unary_marginals = unary_marginals.reshape(unary_potentials.shape)
        y = (unary_marginals, pairwise_marginals)
    else:
        y = np.argmax(unary_marginals, axis=-1)
        y = y.reshape(shape_org)
    if return_energy:
        return y, energy
    return y



In [ ]:

import numpy as np
from joblib import Parallel, delayed
from sklearn.base import BaseEstimator



class BaseSSVM(BaseEstimator):
    """ABC that implements common functionality."""
    def __init__(self, model, max_iter=100, C=1.0, verbose=0,
                 n_jobs=1, show_loss_every=0, logger=None):
        self.model = model
        self.max_iter = max_iter
        self.C = C
        self.verbose = verbose
        self.show_loss_every = show_loss_every
        self.n_jobs = n_jobs
        self.logger = logger

    def predict(self, X, constraints=None):
        """Predict output on examples in X.

        Parameters
        ----------
        X : iterable
            Traing instances. Contains the structured input objects.

        constraints : None or a list of hard logic constraints

        Returns
        -------
        Y_pred : list
            List of inference results for X using the learned parameters.

        """
        verbose = max(0, self.verbose - 3)
        if self.n_jobs != 1:
            if constraints:
                prediction = Parallel(n_jobs=self.n_jobs, verbose=verbose)(
                    delayed(inference)(self.model, x, self.w, constraints=c)
                    for x, c in zip(X, constraints))
            else:
                prediction = Parallel(n_jobs=self.n_jobs, verbose=verbose)(
                    delayed(inference)(self.model, x, self.w) for x in X)
            return prediction
        else:
            if hasattr(self.model, 'batch_inference'):
                if constraints:
                    return self.model.batch_inference(X, self.w,
                                                      constraints=constraints)
                else:
                    return self.model.batch_inference(X, self.w)
            if constraints:
                return [self.model.inference(x, self.w, constraints=c)
                        for x, c in zip(X, constraints)]
            return [self.model.inference(x, self.w) for x in X]

    def score(self, X, Y):
        """Compute score as 1 - loss over whole data set.

        Returns the average accuracy (in terms of model.loss)
        over X and Y.

        Parameters
        ----------
        X : iterable
            Evaluation data.

        Y : iterable
            True labels.

        Returns
        -------
        score : float
            Average of 1 - loss over training examples.
        """
        if hasattr(self.model, 'batch_loss'):
            losses = self.model.batch_loss(Y, self.predict(X))
        else:
            losses = [self.model.loss(y, y_pred)
                      for y, y_pred in zip(Y, self.predict(X))]
        max_losses = [self.model.max_loss(y) for y in Y]
        return 1. - np.sum(losses) / float(np.sum(max_losses))

    def _compute_training_loss(self, X, Y, iteration):
        # optionally compute training loss for output / training curve
        if (self.show_loss_every != 0
                and not iteration % self.show_loss_every):
            if not hasattr(self, 'loss_curve_'):
                self.loss_curve_ = []
            display_loss = 1 - self.score(X, Y)
            if self.verbose > 0:
                print("current loss: %f" % (display_loss))
            self.loss_curve_.append(display_loss)

    def _objective(self, X, Y):
        if type(self).__name__ == 'OneSlackSSVM':
            variant = 'one_slack'
        else:
            variant = 'n_slack'
        return objective_primal(self.model, self.w, X, Y, self.C,
                                variant=variant, n_jobs=self.n_jobs)


In [ ]:
class StructuredModel(object):
    """Interface definition for Structured Learners.

    This class defines what is necessary to use the structured svm.
    You have to implement at least joint_feature and inference.
    """
    def __repr__(self):
        return ("%s, size_joint_feature: %d"
                % (type(self).__name__, self.size_joint_feature))

    def __init__(self):
        """Initialize the model.
        Needs to set self.size_joint_feature, the dimensionality of the joint
        features for an instance with labeling (x, y).
        """
        self.size_joint_feature = None

    def _check_size_w(self, w):
        if w.shape != (self.size_joint_feature,):
            raise ValueError("Got w of wrong shape. Expected %s, got %s" %
                             (self.size_joint_feature, w.shape))

    def initialize(self, X, Y):
        # set any data-specific parameters in the model
        pass

    def joint_feature(self, x, y):
        raise NotImplementedError()

    def batch_joint_feature(self, X, Y, Y_true=None):
        joint_feature_ = np.zeros(self.size_joint_feature)
        if getattr(self, 'rescale_C', False):
            for x, y, y_true in zip(X, Y, Y_true):
                joint_feature_ += self.joint_feature(x, y, y_true)
        else:
            for x, y in zip(X, Y):
                joint_feature_ += self.joint_feature(x, y)
        return joint_feature_

    def _loss_augmented_djoint_feature(self, x, y, y_hat, w):
        # debugging only!
        x_loss_augmented = self.loss_augment(x, y, w)
        return (self.joint_feature(x_loss_augmented, y)
                - self.joint_feature(x_loss_augmented, y_hat))

    def inference(self, x, w, relaxed=None, constraints=None):
        raise NotImplementedError()

    def batch_inference(self, X, w, relaxed=None, constraints=None):
        # default implementation of batch inference
        if constraints:
            return [self.inference(x, w, relaxed=relaxed, constraints=c)
                    for x, c in zip(X, constraints)]
        return [self.inference(x, w, relaxed=relaxed)
                for x in X]

    def loss(self, y, y_hat):
        # hamming loss:
        if isinstance(y_hat, tuple):
            return self.continuous_loss(y, y_hat[0])
        if hasattr(self, 'class_weight'):
            return np.sum(self.class_weight[y] * (y != y_hat))
        return np.sum(y != y_hat)

    def batch_loss(self, Y, Y_hat):
        # default implementation of batch loss
        return [self.loss(y, y_hat) for y, y_hat in zip(Y, Y_hat)]

    def max_loss(self, y):
        # maximum possible los on y for macro averages
        if hasattr(self, 'class_weight'):
            return np.sum(self.class_weight[y])
        return y.size

    def continuous_loss(self, y, y_hat):
        # continuous version of the loss
        # y is the result of linear programming
        if y.ndim == 2:
            raise ValueError("FIXME!")
        gx = np.indices(y.shape)

        # all entries minus correct ones
        result = 1 - y_hat[gx, y]
        if hasattr(self, 'class_weight'):
            return np.sum(self.class_weight[y] * result)
        return np.sum(result)

    def loss_augmented_inference(self, x, y, w, relaxed=None):
        print("FALLBACK no loss augmented inference found")
        return self.inference(x, w)

    def batch_loss_augmented_inference(self, X, Y, w, relaxed=None):
        # default implementation of batch loss augmented inference
        return [self.loss_augmented_inference(x, y, w, relaxed=relaxed)
                for x, y in zip(X, Y)]

    def _set_class_weight(self):
        if not hasattr(self, 'size_joint_feature'):
            # we are not initialized yet
            return

        if hasattr(self, 'n_labels'):
            n_things = self.n_labels
        else:
            n_things = self.n_states

        if self.class_weight is not None:

            if len(self.class_weight) != n_things:
                raise ValueError("class_weight must have length n_states or"
                                 " be None")
            self.class_weight = np.array(self.class_weight)
            self.uniform_class_weight = False
        else:
            self.class_weight = np.ones(n_things)
            self.uniform_class_weight = True


In [ ]:

from time import time

import numpy as np
import cvxopt
import cvxopt.solvers

from joblib import Parallel, delayed




class NoConstraint(Exception):
    # raised if we can not construct a constraint from cache
    pass


class OneSlackSSVM(BaseSSVM):
    """Structured SVM solver for the 1-slack QP with l1 slack penalty.

    Implements margin rescaled structural SVM using
    the 1-slack formulation and cutting plane method, solved using CVXOPT.
    The optimization is restarted in each iteration.

    Parameters
    ----------
    model : StructuredModel
        Object containing the model structure. Has to implement
        `loss`, `inference` and `loss_augmented_inference`.

    max_iter : int, default=10000
        Maximum number of passes over dataset to find constraints.

    C : float, default=1
        Regularization parameter.

    check_constraints : bool
        Whether to check if the new "most violated constraint" is
        more violated than previous constraints. Helpful for stopping
        and debugging, but costly.

    verbose : int
        Verbosity.

    negativity_constraint : list of ints
        Indices of parmeters that are constraint to be negative.
        This is useful for learning submodular CRFs (inference is formulated
        as maximization in SSVMs, flipping some signs).

    break_on_bad : bool default=False
        Whether to break (start debug mode) when inference was approximate.

    n_jobs : int, default=1
        Number of parallel jobs for inference. -1 means as many as cpus.

    show_loss_every : int, default=0
        Controlls how often the hamming loss is computed (for monitoring
        purposes). Zero means never, otherwise it will be computed very
        show_loss_every'th epoch.

    tol : float, default=1e-3
        Convergence tolerance. If dual objective decreases less than tol,
        learning is stopped. The default corresponds to ignoring the behavior
        of the dual objective and stop only if no more constraints can be
        found.

    inference_cache : int, default=0
        How many results of loss_augmented_inference to cache per sample.
        If > 0 the most violating of the cached examples will be used to
        construct a global constraint. Only if this constraint is not violated,
        inference will be run again. This parameter poses a memory /
        computation tradeoff. Storing more constraints might lead to RAM being
        exhausted. Using inference_cache > 0 is only advisable if computation
        time is dominated by inference.

    cache_tol : float, None or 'auto' default='auto'
        Tolerance when to reject a constraint from cache (and do inference).
        If None, ``tol`` will be used. Higher values might lead to faster
        learning. 'auto' uses a heuristic to determine the cache tolerance
        based on the duality gap, as described in [3].

    inactive_threshold : float, default=1e-5
        Threshold for dual variable of a constraint to be considered inactive.

    inactive_window : float, default=50
        Window for measuring inactivity. If a constraint is inactive for
        ``inactive_window`` iterations, it will be pruned from the QP.
        If set to 0, no constraints will be removed.

    switch_to : None or string, default=None
        Switch to the given inference method if the previous method does not
        find any more constraints.

    logger : logger object, default=None
        Pystruct logger for storing the model or extracting additional
        information.

    Attributes
    ----------
    w : nd-array, shape=(model.size_joint_feature,)
        The learned weights of the SVM.

    old_solution : dict
        The last solution found by the qp solver.

    ``loss_curve_`` : list of float
        List of loss values if show_loss_every > 0.

    ``objective_curve_`` : list of float
       Cutting plane objective after each pass through the dataset.

    ``primal_objective_curve_`` : list of float
        Primal objective after each pass through the dataset.

    ``timestamps_`` : list of int
       Total training time stored before each iteration.

    References
    ----------
    [1] Thorsten Joachims, and Thomas Finley and Chun-Nam John Yu:
        Cutting-plane training of structural SVMs, JMLR 2009

    [2] Andreas Mueller: Methods for Learning Structured Prediction in
        Semantic Segmentation of Natural Images, PhD Thesis.  2014

    [3] Andreas Mueller and Sven Behnke: Learning a Loopy Model For Semantic
        Segmentation Exactly, VISAPP 2014

    """

    def __init__(self, model, max_iter=10000, C=1.0, check_constraints=False,
                 verbose=0, negativity_constraint=None, n_jobs=1,
                 break_on_bad=False, show_loss_every=0, tol=1e-3,
                 inference_cache=0, inactive_threshold=1e-5,
                 inactive_window=50, logger=None, cache_tol='auto',
                 switch_to=None):

        BaseSSVM.__init__(self, model, max_iter, C, verbose=verbose,
                          n_jobs=n_jobs, show_loss_every=show_loss_every,
                          logger=logger)

        self.negativity_constraint = negativity_constraint
        self.check_constraints = check_constraints
        self.break_on_bad = break_on_bad
        self.tol = tol
        self.cache_tol = cache_tol
        self.inference_cache = inference_cache
        self.inactive_threshold = inactive_threshold
        self.inactive_window = inactive_window
        self.switch_to = switch_to

    def _solve_1_slack_qp(self, constraints, n_samples):
        C = np.float(self.C) * n_samples  # this is how libsvm/svmstruct do it
        joint_features = [c[0] for c in constraints]
        losses = [c[1] for c in constraints]

        joint_feature_matrix = np.vstack(joint_features)
        n_constraints = len(joint_features)
        P = cvxopt.matrix(np.dot(joint_feature_matrix, joint_feature_matrix.T))
        # q contains loss from margin-rescaling
        q = cvxopt.matrix(-np.array(losses, dtype=np.float))
        # constraints: all alpha must be >zero
        idy = np.identity(n_constraints)
        tmp1 = np.zeros(n_constraints)
        # positivity constraints:
        if self.negativity_constraint is None:
            # empty constraints
            zero_constr = np.zeros(0)
            joint_features_constr = np.zeros((0, n_constraints))
        else:
            joint_features_constr = joint_feature_matrix.T[self.negativity_constraint]
            zero_constr = np.zeros(len(self.negativity_constraint))

        # put together
        G = cvxopt.sparse(cvxopt.matrix(np.vstack((-idy, joint_features_constr))))
        h = cvxopt.matrix(np.hstack((tmp1, zero_constr)))

        # equality constraint: sum of all alpha must be = C
        A = cvxopt.matrix(np.ones((1, n_constraints)))
        b = cvxopt.matrix([C])

        # solve QP model
        cvxopt.solvers.options['feastol'] = 1e-5
        try:
            solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        except ValueError:
            solution = {'status': 'error'}
        if solution['status'] != "optimal":
            print("regularizing QP!")
            P = cvxopt.matrix(np.dot(joint_feature_matrix, joint_feature_matrix.T)
                              + 1e-8 * np.eye(joint_feature_matrix.shape[0]))
            solution = cvxopt.solvers.qp(P, q, G, h, A, b)
            if solution['status'] != "optimal":
                raise ValueError("QP solver failed. Try regularizing your QP.")

        # Lagrange multipliers
        a = np.ravel(solution['x'])
        self.old_solution = solution
        self.prune_constraints(constraints, a)

        # Support vectors have non zero lagrange multipliers
        sv = a > self.inactive_threshold * C
        if self.verbose > 1:
            print("%d support vectors out of %d points" % (np.sum(sv),
                                                           n_constraints))
        self.w = np.dot(a, joint_feature_matrix)
        # we needed to flip the sign to make the dual into a minimization
        # model
        return -solution['primal objective']

    def prune_constraints(self, constraints, a):
        # append list for new constraint
        self.alphas.append([])
        assert(len(self.alphas) == len(constraints))
        for constraint, alpha in zip(self.alphas, a):
            constraint.append(alpha)
            constraint = constraint[-self.inactive_window:]

        # prune unused constraints:
        # if the max of alpha in last 50 iterations was small, throw away
        if self.inactive_window != 0:
            max_active = [np.max(constr[-self.inactive_window:])
                          for constr in self.alphas]
            # find strongest constraint that is not ground truth constraint
            strongest = np.max(max_active[1:])
            inactive = np.where(max_active
                                < self.inactive_threshold * strongest)[0]

            for idx in reversed(inactive):
                # if we don't reverse, we'll mess the indices up
                del constraints[idx]
                del self.alphas[idx]

    def _check_bad_constraint(self, violation, djoint_feature_mean, loss,
                              old_constraints, break_on_bad, tol=None):
        violation_difference = violation - self.last_slack_
        if self.verbose > 1:
            print("New violation: %f difference to last: %f"
                  % (violation, violation_difference))
        if violation_difference < 0 and violation > 0 and break_on_bad:
            raise ValueError("Bad inference: new violation is smaller than"
                             " old.")
        if tol is None:
            tol = self.tol
        if violation_difference < tol:
            if self.verbose:
                print("new constraint too weak.")
            return True
        equals = [True for djoint_feature_, loss_ in old_constraints
                  if (np.all(djoint_feature_ == djoint_feature_mean) and loss == loss_)]

        if np.any(equals):
            return True

        if self.check_constraints:
            for con in old_constraints:
                # compute violation for old constraint
                violation_tmp = max(con[1] - np.dot(self.w, con[0]), 0)
                if self.verbose > 5:
                    print("violation old constraint: %f" % violation_tmp)
                # if violation of new constraint is smaller or not
                # significantly larger, don't add constraint.
                # if smaller, complain about approximate inference.
                if violation - violation_tmp < -1e-5:
                    if self.verbose:
                        print("bad inference: %f" % (violation_tmp - violation))
                    if break_on_bad:
                        raise ValueError("Bad inference: new violation is"
                                         " weaker than previous constraint.")
                    return True
        return False

    @classmethod
    def constraint_equal(cls, y_1, y_2):
        """
        This now more complex. y_1 and/or y_2 (I think) can be: array, pair of
        arrays, pair of list of arrays (multitype)
        We need to compare those!
        """
        if isinstance(y_1, tuple):
            # y_1 is relaxed Y
            # y_1 and y_2 might be lists of ndarray (multitype) instead of
            #    ndarray (single type)
            u_m_1, pw_m_1 = y_1
            if isinstance(y_2, tuple):  # we then compare two relaxed Ys
                u_m_2, pw_m_2 = y_2
                # now, do we multitype or single type relaxed marginals??
                if isinstance(u_m_1, list):
                    return all(np.all(_um1 == _um2) for _um1, _um2
                               in zip( u_m_1,  u_m_2)) \
                        and all(np.all(_pw1 == _pw2) for _pw1, _pw2
                                in zip(pw_m_1, pw_m_2))
                else:
                    return np.all(u_m_1 == u_m_2) and np.all(pw_m_1, pw_m_2)
            else:
                # NOTE original code was possibly comparing array and scalar
                # return np.all(y_1[0] == y_2[0]) and np.all(y_1[1] == y_2[1])
                return False
        # might compare array and tuple... :-/  Was like that, I keep
        return np.all(y_1 == y_2)

    def _update_cache(self, X, Y, Y_hat):
        """Updated cached constraints."""
        if self.inference_cache == 0:
            return
        if (not hasattr(self, "inference_cache_")
                or self.inference_cache_ is None):
            self.inference_cache_ = [[] for y in Y_hat]

        for sample, x, y, y_hat in zip(self.inference_cache_, X, Y, Y_hat):
            already_there = [self.constraint_equal(y_hat, cache[2])
                             for cache in sample]
            if np.any(already_there):
                continue
            if len(sample) > self.inference_cache:
                sample.pop(0)
            # we computed both of these before, but summed them up immediately
            # this makes it a little less efficient in the caching case.
            # the idea is that if we cache, inference is way more expensive
            # and this doesn't matter much.
            sample.append((self.model.joint_feature(x, y_hat),
                           self.model.loss(y, y_hat), y_hat))

    def _constraint_from_cache(self, X, Y, joint_feature_gt, constraints):
        if (not getattr(self, 'inference_cache_', False) or
                self.inference_cache_ is False):
            if self.verbose > 10:
                print("Empty cache.")
            raise NoConstraint
        gap = self.primal_objective_curve_[-1] - self.objective_curve_[-1]
        if (self.cache_tol == 'auto' and gap < self.cache_tol_):
            # do inference if gap has become to small
            if self.verbose > 1:
                print("Last gap too small (%f < %f), not loading constraint"
                      " from cache."
                      % (gap, self.cache_tol_))
            raise NoConstraint

        Y_hat = []
        joint_feature_acc = np.zeros(self.model.size_joint_feature)
        loss_mean = 0
        for cached in self.inference_cache_:
            # cached has entries of form (joint_feature, loss, y_hat)
            violations = [np.dot(joint_feature, self.w) + loss
                          for joint_feature, loss, _ in cached]
            joint_feature, loss, y_hat = cached[np.argmax(violations)]
            Y_hat.append(y_hat)
            joint_feature_acc += joint_feature
            loss_mean += loss

        djoint_feature = (joint_feature_gt - joint_feature_acc) / len(X)
        loss_mean = loss_mean / len(X)

        violation = loss_mean - np.dot(self.w, djoint_feature)
        if self._check_bad_constraint(violation, djoint_feature, loss_mean, constraints,
                                      break_on_bad=False):
            if self.verbose > 1:
                print("No constraint from cache.")
            raise NoConstraint
        return Y_hat, djoint_feature, loss_mean

    def _find_new_constraint(self, X, Y, joint_feature_gt, constraints, check=True):
        if self.n_jobs != 1:
            # do inference in parallel
            verbose = max(0, self.verbose - 3)
            Y_hat = Parallel(n_jobs=self.n_jobs, verbose=verbose)(
                delayed(loss_augmented_inference)(
                    self.model, x, y, self.w, relaxed=True)
                for x, y in zip(X, Y))
        else:
            Y_hat = self.model.batch_loss_augmented_inference(
                X, Y, self.w, relaxed=True)
        # compute the mean over joint_features and losses

        if getattr(self.model, 'rescale_C', False):
            djoint_feature = (joint_feature_gt
                              - self.model.batch_joint_feature(X, Y_hat, Y)) / len(X)
        else:
            djoint_feature = (joint_feature_gt
                              - self.model.batch_joint_feature(X, Y_hat)) / len(X)

        loss_mean = np.mean(self.model.batch_loss(Y, Y_hat))

        violation = loss_mean - np.dot(self.w, djoint_feature)
        if check and self._check_bad_constraint(
                violation, djoint_feature, loss_mean, constraints,
                break_on_bad=self.break_on_bad):
            raise NoConstraint
        return Y_hat, djoint_feature, loss_mean

    def fit(self, X, Y, constraints=None, warm_start=False, initialize=True):
        """Learn parameters using cutting plane method.

        Parameters
        ----------
        X : iterable
            Traing instances. Contains the structured input objects.
            No requirement on the particular form of entries of X is made.

        Y : iterable
            Training labels. Contains the strctured labels for inputs in X.
            Needs to have the same length as X.

        contraints : ignored

        warm_start : bool, default=False
            Whether we are warmstarting from a previous fit.

        initialize : boolean, default=True
            Whether to initialize the model for the data.
            Leave this true except if you really know what you are doing.
        """
        if self.verbose:
            print("Training 1-slack dual structural SVM")
        cvxopt.solvers.options['show_progress'] = self.verbose > 3
        if initialize:
            self.model.initialize(X, Y)

        # parse cache_tol parameter
        if self.cache_tol is None or self.cache_tol == 'auto':
            self.cache_tol_ = self.tol
        else:
            self.cache_tol_ = self.cache_tol

        if not warm_start:
            self.w = np.zeros(self.model.size_joint_feature)
            constraints = []
            self.objective_curve_, self.primal_objective_curve_ = [], []
            self.cached_constraint_ = []
            self.alphas = []  # dual solutions
            # append constraint given by ground truth to make our life easier
            constraints.append((np.zeros(self.model.size_joint_feature), 0))
            self.alphas.append([self.C])
            self.inference_cache_ = None
            self.timestamps_ = [time()]
        elif warm_start == "soft":
            self.w = np.zeros(self.model.size_joint_feature)
            constraints = []
            self.alphas = []  # dual solutions
            # append constraint given by ground truth to make our life easier
            constraints.append((np.zeros(self.model.size_joint_feature), 0))
            self.alphas.append([self.C])

        else:
            constraints = self.constraints_

        self.last_slack_ = -1

        # get the joint_feature of the ground truth
        if getattr(self.model, 'rescale_C', False):
            joint_feature_gt = self.model.batch_joint_feature(X, Y, Y)
        else:
            joint_feature_gt = self.model.batch_joint_feature(X, Y)

        try:
            # catch ctrl+c to stop training

            for iteration in range(self.max_iter):
                # main loop
                cached_constraint = False
                if self.verbose > 0:
                    print("iteration %d" % iteration)
                if self.verbose > 2:
                    print(self)
                try:
                    Y_hat, djoint_feature, loss_mean = self._constraint_from_cache(
                        X, Y, joint_feature_gt, constraints)
                    cached_constraint = True
                except NoConstraint:
                    try:
                        Y_hat, djoint_feature, loss_mean = self._find_new_constraint(
                            X, Y, joint_feature_gt, constraints)
                        self._update_cache(X, Y, Y_hat)
                    except NoConstraint:
                        if self.verbose:
                            print("no additional constraints")
                        if (self.switch_to is not None
                                and self.model.inference_method !=
                                self.switch_to):
                            if self.verbose:
                                print("Switching to %s inference" %
                                      str(self.switch_to))
                            self.model.inference_method_ = \
                                self.model.inference_method
                            self.model.inference_method = self.switch_to
                            continue
                        else:
                            break

                self.timestamps_.append(time() - self.timestamps_[0])
                self._compute_training_loss(X, Y, iteration)
                constraints.append((djoint_feature, loss_mean))

                # compute primal objective
                last_slack = -np.dot(self.w, djoint_feature) + loss_mean
                primal_objective = (self.C * len(X)
                                    * max(last_slack, 0)
                                    + np.sum(self.w ** 2) / 2)
                self.primal_objective_curve_.append(primal_objective)
                self.cached_constraint_.append(cached_constraint)

                objective = self._solve_1_slack_qp(constraints,
                                                   n_samples=len(X))

                # update cache tolerance if cache_tol is auto:
                if self.cache_tol == "auto" and not cached_constraint:
                    self.cache_tol_ = (primal_objective - objective) / 4

                self.last_slack_ = np.max([(-np.dot(self.w, djoint_feature) + loss_mean)
                                           for djoint_feature, loss_mean in constraints])
                self.last_slack_ = max(self.last_slack_, 0)

                if self.verbose > 0:
                    # the cutting plane objective can also be computed as
                    # self.C * len(X) * self.last_slack_ + np.sum(self.w**2)/2
                    print("cutting plane objective: %f, primal objective %f"
                          % (objective, primal_objective))
                # we only do this here because we didn't add the gt to the
                # constraints, which makes the dual behave a bit oddly
                self.objective_curve_.append(objective)
                self.constraints_ = constraints
                if self.logger is not None:
                    self.logger(self, iteration)

                if self.verbose > 5:
                    print(self.w)
        except KeyboardInterrupt:
            pass
        if self.verbose and self.n_jobs == 1:
            print("calls to inference: %d" % self.model.inference_calls)
        # compute final objective:
        self.timestamps_.append(time() - self.timestamps_[0])
        primal_objective = self._objective(X, Y)
        self.primal_objective_curve_.append(primal_objective)
        self.objective_curve_.append(objective)
        self.cached_constraint_.append(False)

        if self.logger is not None:
            self.logger(self, 'final')

        if self.verbose > 0:
            print("final primal objective: %f gap: %f"
                  % (primal_objective, primal_objective - objective))

        return self


In [ ]:
from time import time
import numpy as np

from joblib import Parallel, delayed, cpu_count
from sklearn.utils import gen_even_slices, shuffle



class SubgradientSSVM(BaseSSVM):
    """Structured SVM solver using subgradient descent.

    Implements a margin rescaled with l1 slack penalty.
    By default, a constant learning rate is used.
    It is also possible to use the adaptive learning rate found by AdaGrad.

    This class implements online subgradient descent. If n_jobs != 1,
    small batches of size n_jobs are used to exploit parallel inference.
    If inference is fast, use n_jobs=1.

    Parameters
    ----------
    model : StructuredModel
        Object containing model structure. Has to implement
        `loss`, `inference` and `loss_augmented_inference`.

    max_iter : int, default=100
        Maximum number of passes over dataset to find constraints and perform
        updates.

    C : float, default=1.
        Regularization parameter.

    verbose : int, default=0
        Verbosity.

    learning_rate : float or 'auto', default='auto'
        Learning rate used in subgradient descent. If 'auto', the pegasos
        schedule is used, which starts with ``learning_rate = n_samples * C``.

    momentum : float, default=0.0
        Momentum used in subgradient descent.

    n_jobs : int, default=1
        Number of parallel jobs for inference. -1 means as many as cpus.

    batch_size : int, default=None
        Ignored if n_jobs > 1. If n_jobs=1, inference will be done in mini
        batches of size batch_size. If n_jobs=-1, batch learning will be
        performed, that is the whole dataset will be used to compute each
        subgradient.

    show_loss_every : int, default=0
        Controlls how often the hamming loss is computed (for monitoring
        purposes). Zero means never, otherwise it will be computed very
        show_loss_every'th epoch.

    decay_exponent : float, default=1
        Exponent for decaying learning rate. Effective learning rate is
        ``learning_rate / (decay_t0 + t)** decay_exponent``. Zero means no
        decay.

    decay_t0 : float, default=10
        Offset for decaying learning rate. Effective learning rate is
        ``learning_rate / (decay_t0 + t)** decay_exponent``.

    break_on_no_constraints : bool, default=True
        Break when there are no new constraints found.

    logger : logger object.

    averaging : string, default=None
        Whether and how to average weights. Possible options are 'linear',
        'squared' and None.
        The string reflects the weighting of the averaging:

            - ``linear: w_avg ~ w_1 + 2 * w_2 + ... + t * w_t``

            - ``squared: w_avg ~ w_1 + 4 * w_2 + ... + t**2 * w_t``

        Uniform averaging is not implemented as it is worse than linear
        weighted averaging or no averaging.

    shuffle : bool, default=False
        Whether to shuffle the dataset in each iteration.

    Attributes
    ----------
    w : nd-array, shape=(model.size_joint_feature,)
        The learned weights of the SVM.

    ``loss_curve_`` : list of float
        List of loss values if show_loss_every > 0.

    ``objective_curve_`` : list of float
       Primal objective after each pass through the dataset.

    ``timestamps_`` : list of int
       Total training time stored before each iteration.

    References
    ----------
    * Nathan Ratliff, J. Andrew Bagnell and Martin Zinkevich:
        (Online) Subgradient Methods for Structured Prediction, AISTATS 2007

    * Shalev-Shwartz, Shai and Singer, Yoram and Srebro, Nathan and Cotter,
        Andrew: Pegasos: Primal estimated sub-gradient solver for svm,
        Mathematical Programming 2011
    """
    def __init__(self, model, max_iter=100, C=1.0, verbose=0, momentum=0.0,
                 learning_rate='auto', n_jobs=1,
                 show_loss_every=0, decay_exponent=1,
                 break_on_no_constraints=True, logger=None, batch_size=None,
                 decay_t0=10, averaging=None, shuffle=False):
        BaseSSVM.__init__(self, model, max_iter, C, verbose=verbose,
                          n_jobs=n_jobs, show_loss_every=show_loss_every,
                          logger=logger)
        self.averaging = averaging
        self.break_on_no_constraints = break_on_no_constraints
        self.momentum = momentum
        self.learning_rate = learning_rate
        self.t = 0
        self.decay_exponent = decay_exponent
        self.decay_t0 = decay_t0
        self.batch_size = batch_size
        self.shuffle = shuffle

    def _solve_subgradient(self, djoint_feature, n_samples, w):
        """Do a single subgradient step."""
        grad = (djoint_feature - w / (self.C * n_samples))

        self.grad_old = ((1 - self.momentum) * grad
                         + self.momentum * self.grad_old)
        if self.decay_exponent == 0:
            effective_lr = self.learning_rate_
        else:
            effective_lr = (self.learning_rate_
                            / (self.t + self.decay_t0)
                            ** self.decay_exponent)
        w += effective_lr * self.grad_old

        if self.averaging == 'linear':
            rho = 2. / (self.t + 2.)
            self.w = (1. - rho) * self.w + rho * w
        elif self.averaging == 'squared':
            rho = 6. * (self.t + 1) / ((self.t + 2) * (2 * self.t + 3))
            self.w = (1. - rho) * self.w + rho * w
        else:
            self.w = w
        self.t += 1.
        return w

    def fit(self, X, Y, constraints=None, warm_start=False, initialize=True):
        """Learn parameters using subgradient descent.

        Parameters
        ----------
        X : iterable
            Traing instances. Contains the structured input objects.
            No requirement on the particular form of entries of X is made.

        Y : iterable
            Training labels. Contains the strctured labels for inputs in X.
            Needs to have the same length as X.

        constraints : None
            Discarded. Only for API compatibility currently.

        warm_start : boolean, default=False
            Whether to restart a previous fit.

        initialize : boolean, default=True
            Whether to initialize the model for the data.
            Leave this true except if you really know what you are doing.
        """
        if initialize:
            self.model.initialize(X, Y)
        if self.verbose:
            print("Training primal subgradient structural SVM")
        self.grad_old = np.zeros(self.model.size_joint_feature)
        self.w = getattr(self, "w", np.zeros(self.model.size_joint_feature))
        w = self.w.copy()
        if not warm_start:
            self.objective_curve_ = []
            self.timestamps_ = [time()]
            if self.learning_rate == "auto":
                self.learning_rate_ = self.C * len(X)
            else:
                self.learning_rate_ = self.learning_rate
        else:
            self.timestamps_ = (np.array(self.timestamps_) - time()).tolist()
        try:
            # catch ctrl+c to stop training
            for iteration in range(self.max_iter):
                if self.shuffle:
                    X, Y = shuffle(X, Y)
                if self.n_jobs == 1:
                    objective, positive_slacks, w = self._sequential_learning(X, Y, w)
                else:
                    objective, positive_slacks, w = self._parallel_learning(X, Y, w)

                # some statistics
                objective = objective * self.C + np.sum(w ** 2) / 2.

                if positive_slacks == 0:
                    if self.verbose:
                        print("No additional constraints")
                    if self.break_on_no_constraints:
                        break
                if self.verbose > 0:
                    print(self)
                    print("iteration %d" % iteration)
                    print("positive slacks: %d,"
                          "objective: %f" %
                          (positive_slacks, objective))
                self.timestamps_.append(time() - self.timestamps_[0])
                self.objective_curve_.append(self._objective(X, Y))

                if self.verbose > 2:
                    print(self.w)

                self._compute_training_loss(X, Y, iteration)
                if self.logger is not None:
                    self.logger(self, iteration)

        except KeyboardInterrupt:
            pass

        if self.verbose:
            print("Computing final objective")

        self.timestamps_.append(time() - self.timestamps_[0])
        self.objective_curve_.append(self._objective(X, Y))
        if self.logger is not None:
            self.logger(self, 'final')
        if self.verbose:
            if self.objective_curve_:
                print("final objective: %f" % self.objective_curve_[-1])
            if self.verbose and self.n_jobs == 1:
                print("calls to inference: %d" % self.model.inference_calls)

        return self

    def _parallel_learning(self, X, Y, w):
        n_samples = len(X)
        objective, positive_slacks = 0, 0
        verbose = max(0, self.verbose - 3)
        if self.batch_size is not None:
            raise ValueError("If n_jobs != 1, batch_size needs to"
                             "be None")
        # generate batches of size n_jobs
        # to speed up inference
        if self.n_jobs == -1:
            n_jobs = cpu_count()
        else:
            n_jobs = self.n_jobs

        n_batches = int(np.ceil(float(len(X)) / n_jobs))
        slices = gen_even_slices(n_samples, n_batches)
        for batch in slices:
            X_b = X[batch]
            Y_b = Y[batch]
            candidate_constraints = Parallel(
                n_jobs=self.n_jobs,
                verbose=verbose)(delayed(find_constraint)(
                    self.model, x, y, w)
                    for x, y in zip(X_b, Y_b))
            djoint_feature = np.zeros(self.model.size_joint_feature)
            for x, y, constraint in zip(X_b, Y_b,
                                        candidate_constraints):
                y_hat, delta_joint_feature, slack, loss = constraint
                if slack > 0:
                    objective += slack
                    djoint_feature += delta_joint_feature
                    positive_slacks += 1
            w = self._solve_subgradient(djoint_feature, n_samples, w)
        return objective, positive_slacks, w

    def _sequential_learning(self, X, Y, w):
        n_samples = len(X)
        objective, positive_slacks = 0, 0
        if self.batch_size in [None, 1]:
            # online learning
            for x, y in zip(X, Y):
                y_hat, delta_joint_feature, slack, loss = \
                    find_constraint(self.model, x, y, w)
                objective += slack
                if slack > 0:
                    positive_slacks += 1
                self._solve_subgradient(delta_joint_feature, n_samples, w)
        else:
            # mini batch learning
            if self.batch_size == -1:
                slices = [slice(0, len(X))]
            else:
                n_batches = int(np.ceil(float(len(X)) / self.batch_size))
                slices = gen_even_slices(n_samples, n_batches)
            for batch in slices:
                X_b = X[batch]
                Y_b = Y[batch]
                Y_hat = self.model.batch_loss_augmented_inference(
                    X_b, Y_b, w, relaxed=True)
                delta_joint_feature = (self.model.batch_joint_feature(X_b, Y_b)
                                       - self.model.batch_joint_feature(X_b, Y_hat))
                loss = np.sum(self.model.batch_loss(Y_b, Y_hat))

                violation = np.maximum(0, loss - np.dot(w, delta_joint_feature))
                objective += violation
                positive_slacks += self.batch_size
                self._solve_subgradient(delta_joint_feature / len(X_b), n_samples, w)
        return objective, positive_slacks, w


In [ ]:
class CRF(StructuredModel):
    """Abstract base class"""
    def __init__(self, n_states=None, n_features=None, inference_method=None,
                 class_weight=None):
        self.n_states = n_states
        if inference_method is None:
            # get first in list that is installed
            inference_method = get_installed(['ad3', 'max-product', 'lp'])[0]
        self.inference_method = inference_method
        self.inference_calls = 0
        self.n_features = n_features
        self.class_weight = class_weight
        self._set_size_joint_feature()
        self._set_class_weight()

    def initialize(self, X, Y):
        # Works for both GridCRF and GraphCRF, but not ChainCRF.
        # funny that ^^
        n_features = X[0][0].shape[1]
        if self.n_features is None:
            self.n_features = n_features
        elif self.n_features != n_features:
            raise ValueError("Expected %d features, got %d"
                             % (self.n_features, n_features))

        n_states = len(np.unique(np.hstack([y.ravel() for y in Y])))
        if self.n_states is None:
            self.n_states = n_states
        elif self.n_states != n_states:
            raise ValueError("Expected %d states, got %d"
                             % (self.n_states, n_states))

        self._set_size_joint_feature()
        self._set_class_weight()

    def __repr__(self):
        return ("%s(n_states: %s, inference_method: %s)"
                % (type(self).__name__, self.n_states,
                   self.inference_method))

    def _check_size_x(self, x):
        features = self._get_features(x)
        if features.shape[1] != self.n_features:
            raise ValueError("Unary evidence should have %d feature per node,"
                             " got %s instead."
                             % (self.n_features, features.shape[1]))

    def loss_augment_unaries(self, unary_potentials, y):
        """
        we define it as a method so that subclasses can specialize it.
        """
        loss_augment_unaries(unary_potentials, np.asarray(y),
                             self.class_weight)

    def loss_augmented_inference(self, x, y, w, relaxed=False,
                                 return_energy=False):
        """Loss-augmented Inference for x relative to y using parameters w.

        Finds (approximately)
        armin_y_hat np.dot(w, joint_feature(x, y_hat)) + loss(y, y_hat)
        using self.inference_method.


        Parameters
        ----------
        x : tuple
            Instance of a graph with unary evidence.
            x=(unaries, edges)
            unaries are an nd-array of shape (n_nodes, n_features),
            edges are an nd-array of shape (n_edges, 2)

        y : ndarray, shape (n_nodes,)
            Ground truth labeling relative to which the loss
            will be measured.

        w : ndarray, shape=(size_joint_feature,)
            Parameters for the CRF energy function.

        relaxed : bool, default=False
            Whether relaxed inference should be performed.
            Only meaningful if inference method is 'lp' or 'ad3'.
            By default fractional solutions are rounded. If relaxed=True,
            fractional solutions are returned directly.

        return_energy : bool, default=False
            Whether to return the energy of the solution (x, y) that was found.

        Returns
        -------
        y_pred : ndarray or tuple
            By default an inter ndarray of shape=(n_nodes)
            of variable assignments for x is returned.
            If ``relaxed=True`` and inference_method is ``lp`` or ``ad3``,
            a tuple (unary_marginals, pairwise_marginals)
            containing the relaxed inference result is returned.
            unary marginals is an array of shape (n_nodes, n_states),
            pairwise_marginals is an array of
            shape (n_states, n_states) of accumulated pairwise marginals.

        """
        self.inference_calls += 1
        self._check_size_w(w)
        unary_potentials = self._get_unary_potentials(x, w)
        pairwise_potentials = self._get_pairwise_potentials(x, w)
        edges = self._get_edges(x)

        self.loss_augment_unaries(unary_potentials, y)

        return inference_dispatch(unary_potentials, pairwise_potentials, edges,
                                  self.inference_method, relaxed=relaxed,
                                  return_energy=return_energy)

    def inference(self, x, w, relaxed=False, return_energy=False,
                  constraints=None):
        """Inference for x using parameters w.

        Finds (approximately)
        armin_y np.dot(w, joint_feature(x, y))
        using self.inference_method.


        Parameters
        ----------
        x : tuple
            Instance of a graph with unary evidence.
            x=(unaries, edges)
            unaries are an nd-array of shape (n_nodes, n_states),
            edges are an nd-array of shape (n_edges, 2)

        w : ndarray, shape=(size_joint_feature,)
            Parameters for the CRF energy function.

        relaxed : bool, default=False
            Whether relaxed inference should be performed.
            Only meaningful if inference method is 'lp' or 'ad3'.
            By default fractional solutions are rounded. If relaxed=True,
            fractional solutions are returned directly.

        return_energy : bool, default=False
            Whether to return the energy of the solution (x, y) that was found.

        constraints : None or list, default=False
            hard logic constraints, if any

        Returns
        -------
        y_pred : ndarray or tuple
            By default an inter ndarray of shape=(width, height)
            of variable assignments for x is returned.
            If ``relaxed=True`` and inference_method is ``lp`` or ``ad3``,
            a tuple (unary_marginals, pairwise_marginals)
            containing the relaxed inference result is returned.
            unary marginals is an array of shape (width, height, n_states),
            pairwise_marginals is an array of
            shape (n_states, n_states) of accumulated pairwise marginals.

        """
        self._check_size_w(w)
        self.inference_calls += 1
        unary_potentials = self._get_unary_potentials(x, w)
        pairwise_potentials = self._get_pairwise_potentials(x, w)
        edges = self._get_edges(x)

        if constraints:
            return inference_dispatch(unary_potentials, pairwise_potentials,
                                      edges,
                                      self.inference_method,
                                      relaxed=relaxed,
                                      return_energy=return_energy,
                                      constraints=constraints)
        else:
            return inference_dispatch(unary_potentials, pairwise_potentials,
                                      edges,
                                      self.inference_method,
                                      relaxed=relaxed,
                                      return_energy=return_energy)


In [ ]:
class MultiLabelClf(CRF):
    """Multi-label model for predicting several binary classes.

    Multi-label classification is a generalization of multi-class
    classification, in that multiple classes can be present in each
    example. This can also be thought of as predicting
    binary indicator per class.

    This class supports different models via the "edges" parameter.
    Giving no edges yields independent classifiers for each class. Giving
    "full" yields a fully connected graph over the labels, while "tree"
    yields the best tree-shaped graph (using the Chow-Liu algorithm).
    It is also possible to specify a custom connectivity structure.

    Parameters
    ----------
    n_labels : int (default=None)
        Number of labels. Inferred from data if not provided.

    n_features : int (default=None)
        Number of input features. Inferred from data if not provided.

    edges : array-like, string or None
        Either None, which yields independent models, 'tree',
        which yields the Chow-Liu tree over the labels, 'full',
        which yields a fully connected graph, or an array-like
        of edges for a custom dependency structure.

    inference_method :
        The inference method to be used.

    """
    def __init__(self, n_labels=None, n_features=None, edges=None,
                 inference_method=None):
        self.n_labels = n_labels
        self.edges = edges
        CRF.__init__(self, 2, n_features, inference_method)

    def _set_size_joint_feature(self):
        # try to set the size of joint_feature if possible
        if self.n_features is not None and self.n_states is not None:
            if self.edges is None:
                self.edges = np.zeros(shape=(0, 2), dtype=np.int)
            self.size_joint_feature = (self.n_features * self.n_labels + 4 *
                                       self.edges.shape[0])

    def initialize(self, X, Y):
        n_features = X.shape[1]
        if self.n_features is None:
            self.n_features = n_features
        elif self.n_features != n_features:
            raise ValueError("Expected %d features, got %d"
                             % (self.n_features, n_features))

        n_labels = Y.shape[1]
        if self.n_labels is None:
            self.n_labels = n_labels
        elif self.n_labels != n_labels:
            raise ValueError("Expected %d labels, got %d"
                             % (self.n_labels, n_labels))

        self._set_size_joint_feature()
        self._set_class_weight()

    def _get_edges(self, x):
        return self.edges

    def _get_unary_potentials(self, x, w):
        unary_params = w[:self.n_labels * self.n_features].reshape(
            self.n_labels, self.n_features)
        unary_potentials = np.dot(x, unary_params.T)
        return np.vstack([-unary_potentials, unary_potentials]).T

    def _get_pairwise_potentials(self, x, w):
        pairwise_params = w[self.n_labels * self.n_features:].reshape(
            self.edges.shape[0], self.n_states, self.n_states)
        return pairwise_params

    def joint_feature(self, x, y):
        if isinstance(y, tuple):
            # continuous pairwise marginals
            y_cont, pairwise_marginals = y
            y_signs = 2 * y_cont[:, 1] - 1
            unary_marginals = np.repeat(x[np.newaxis, :], len(y_signs), axis=0)
            unary_marginals *= y_signs[:, np.newaxis]
        else:
            # discrete y
            y_signs = 2 * y - 1
            unary_marginals = np.repeat(x[np.newaxis, :], len(y_signs), axis=0)
            unary_marginals *= y_signs[:, np.newaxis]
            pairwise_marginals = []
            for edge in self.edges:
                # indicator of one of four possible states of the edge
                pw = np.zeros((2, 2))
                pw[y[edge[0]], y[edge[1]]] = 1
                pairwise_marginals.append(pw)

        if len(pairwise_marginals):
            pairwise_marginals = np.vstack(pairwise_marginals)
            return np.hstack([unary_marginals.ravel(),
                              pairwise_marginals.ravel()])
        return unary_marginals.ravel()


In [ ]:
from os.path import dirname, join
import sys

try:
    import cPickle as pickle
except ImportError:
    import pickle

import numpy as np


def _safe_unpickle(file_name):
    with open(file_name, "rb") as data_file:
        if sys.version_info >= (3, 0):
            # python3 unpickling of python2 unicode
            data = pickle.load(data_file, encoding="latin1")
        else:
            data = pickle.load(data_file)
    return data


def load_letters():
    """Load the OCR letters dataset.

    This is a chain classification task.
    Each example consists of a word, segmented into letters.
    The first letter of each word is ommited from the data,
    as it was a capital letter (in contrast to all other letters).


    References
    ----------
    http://papers.nips.cc/paper/2397-max-margin-markov-networks.pdf
    http://groups.csail.mit.edu/sls/archives/root/publications/1995/Kassel%20Thesis.pdf
    http://www.seas.upenn.edu/~taskar/ocr/
    """
    #module_path = dirname(__file__)
    #module_path = r"C:\Users\רוני\OneDrive‏ - Technion\Documents\סמסטר ה\למידה 2\PROJ\pystruct-master\pystruct\datasets"
    #module_path = r"C:\Users\noamg\Documents\school\technion\doctorat courses\ML2\project\pythonProject"
    module_path="/"

    data = _safe_unpickle(join(module_path, 'letters.pickle'))
    # we add an easy to use image representation:
    data['images'] = [np.hstack([l.reshape(16, 8) for l in word])
                      for word in data['data']]
    return data


def load_scene():
    """Load the scene multi-label dataset.

    This is a benchmark multilabel dataset.
    n_classes = 6
    n_fetures = 294
    n_samples_test = 1196
    n_samples_train = 1211

    References
    ----------
    Matthew R. Boutell, Jiebo Luo, Xipeng Shen, and Christopher M. Brown.
    Learning multi-label scene classification.
    """
    #module_path = r"C:\Users\רוני\OneDrive‏ - Technion\Documents\סמסטר ה\למידה 2\PROJ\pystruct-master\pystruct\datasets"
   # module_path = r"C:\Users\noamg\Documents\school\technion\doctorat courses\ML2\project\pythonProject"
    module_path="/"
    return _safe_unpickle(join(module_path, 'scene.pickle'))


def load_snakes():
    """Load the synthetic snake datasets.

    Taken from:
    Nowozin, S., Rother, C., Bagon, S., Sharp, T., Yao, B., & Kohli, P.
    Decision Tree Fields, ICCV 2011

    This is a 2d grid labeling task where conditinal pairwise interactions are
    important.
    See the reference for an explanation.
    """
   # module_path = r"C:\Users\רוני\OneDrive‏ - Technion\Documents\סמסטר ה\למידה 2\PROJ\pystruct-master\pystruct\datasets"
    #module_path = r"C:\Users\noamg\Documents\school\technion\doctorat courses\ML2\project\pythonProject"
    #module_path = dirname(__file__)
    module_path="/"
    return _safe_unpickle(join(module_path, 'snakes.pickle'))


In [ ]:
from google.colab import files
scene = files.upload()

Saving scene.pickle to scene.pickle


In [ ]:
scene=load_scene()
x_train_r, x_test_r = scene['X_train'], scene['X_test']
y_train_r, y_test_r = scene['y_train'], scene['y_test']

In [ ]:
#start_1 = time()
model1 = OneSlackSSVM(MultiLabelClf(),max_iter = 1000)
model1.fit(x_train_r, y_train_r)
score_1 = model1.score(x_test_r,y_test_r)
print(f"Old model score {score_1:.3f}")
#print(f"Time for 100 iterations is {time()-start_1}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depreca

Old model score 0.866


In [ ]:
#start_2 = time()
model2 = SubgradientSSVM(MultiLabelClf(),max_iter = 1000,batch_size = 10)
model2.fit(x_train_r, y_train_r)
score_2 = model2.score(x_test_r, y_test_r)
print(f"New model score {score_2:.3f}")
#print(f"Time for 10 epochs is {time()-start_2}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depreca

New model score 0.790
